In [105]:
import pandas as pd
import pickle
import requests

#### Data Preperation

In [106]:
data = pd.read_csv('../data/prepared_data_v3.csv')
no_weeks = data['week_number'].max()
for week in range(no_weeks):
    data.loc[data['week_number'] == week, 'next_week_plays'] = data.loc[data['week_number'] == week+1, 'track_plays'].values

data['next_week_plays'] = data['next_week_plays'].fillna(0).astype(int)
data = data.drop(data[data['week_number'] == no_weeks].index)

In [107]:
def load_week_data(data: pd.DataFrame, week_number: int) -> pd.DataFrame:
    return data[data['week_number'] == week_number].copy()

In [108]:
week_data = load_week_data(data, 1)
next_week_data = load_week_data(data, 2)

In [109]:
next_week_top_tracks  = next_week_data.sort_values(by='track_plays', ascending=False).head(50)

In [110]:
next_week_top_tracks[['track_plays', 'name_track', 'name_artist']]

,track_plays,name_track,name_artist
54223,42,Memories,Maroon 5
58509,40,The Business,Tiësto
55568,37,Snowman,Sia
64382,35,Whoopty,CJ
58639,35,The Nights,Avicii
63061,34,you broke me first,Tate McRae
62828,34,Falling,Harry Styles
64327,34,WITHOUT YOU,The Kid LAROI
55997,32,All of Me,John Legend
62826,31,Golden,Harry Styles


In [111]:
week_data.drop(columns=['week_number', 'name_track', 'name_artist', 'release_date', 'genres'], inplace=True)

In [112]:
week_data_input = week_data.drop('next_week_plays', axis=1).values.tolist()

#### Predictions

In [113]:
torch_res = requests.post('http://localhost:8080/torch-model/top-playlist', json=week_data_input)
xgb_res = requests.post('http://localhost:8080/xgb-model/top-playlist', json=week_data_input)

In [114]:
torch_predicted_top_tracks = next_week_data[next_week_data['id_track'].isin(torch_res.json())]

In [115]:
torch_predicted_top_tracks[['track_plays', 'name_track', 'name_artist']]

,track_plays,name_track,name_artist
53152,23,Yellow,Coldplay
55568,37,Snowman,Sia
55731,28,Cupid's Chokehold / Breakfast in America,Gym Class Heroes
56130,30,Why'd You Only Call Me When You're High?,Arctic Monkeys
57779,23,Hold On,Justin Bieber
57780,25,Anyone,Justin Bieber
58509,40,The Business,Tiësto
58684,24,Shape of You,Ed Sheeran
58693,29,Afterglow,Ed Sheeran
59287,29,Follow You,Imagine Dragons


In [116]:
torch_matching_tracks = next_week_top_tracks[next_week_top_tracks['id_track'].isin(torch_res.json())]

In [117]:
len(torch_matching_tracks)

25

In [118]:
xgb_predicted_top_tracks = next_week_top_tracks[next_week_top_tracks['id_track'].isin(xgb_res.json())]

In [119]:
xgb_predicted_top_tracks[['track_plays', 'name_track', 'name_artist']]

,track_plays,name_track,name_artist
54223,42,Memories,Maroon 5
55568,37,Snowman,Sia
64382,35,Whoopty,CJ
58639,35,The Nights,Avicii
63061,34,you broke me first,Tate McRae
62828,34,Falling,Harry Styles
55997,32,All of Me,John Legend
64384,31,Goosebumps,HVME
63333,31,Lucid Dreams,Juice WRLD
62047,31,everything i wanted,Billie Eilish


In [120]:
xgb_matching_tracks = next_week_top_tracks[next_week_top_tracks['id_track'].isin(xgb_res.json())]

In [121]:
len(xgb_matching_tracks)

32

#### A/B Testing

In [122]:
import numpy as np
from scipy import stats

In [123]:
weeks_to_test = np.random.choice(range(51), size=20, replace=False)

In [124]:
def get_matches_for_week(week_number):
    week_data = load_week_data(data, week_number)
    next_week_data = load_week_data(data, week_number+1)

    next_week_top_tracks  = next_week_data.sort_values(by='track_plays', ascending=False).head(50)

    week_data.drop(columns=['week_number', 'name_track', 'name_artist', 'release_date', 'genres'], inplace=True)

    week_data_input = week_data.drop('next_week_plays', axis=1).values.tolist()

    torch_res = requests.post('http://localhost:8080/torch-model/top-playlist', json=week_data_input)
    xgb_res = requests.post('http://localhost:8080/xgb-model/top-playlist', json=week_data_input)

    torch_matching_tracks = next_week_top_tracks[next_week_top_tracks['id_track'].isin(torch_res.json())]
    xgb_matching_tracks = next_week_top_tracks[next_week_top_tracks['id_track'].isin(xgb_res.json())]

    return len(torch_matching_tracks), len(xgb_matching_tracks)

In [125]:
modelA_matches = []
modelB_matches = []
for week in weeks_to_test:
    modelA, modelB = get_matches_for_week(week)
    modelA_matches.append(modelA)
    modelB_matches.append(modelB)

In [126]:
modelA_matches, modelB_matches

([23,
  23,
  24,
  23,
  24,
  25,
  22,
  21,
  27,
  25,
  24,
  30,
  24,
  24,
  28,
  23,
  26,
  25,
  23,
  26],
 [33,
  30,
  31,
  26,
  25,
  28,
  26,
  30,
  26,
  24,
  25,
  31,
  31,
  27,
  24,
  27,
  29,
  28,
  28,
  28])

In [127]:
t_stat, p_val = stats.ttest_ind(modelA_matches, modelB_matches)

print(f"T-statistic: {t_stat}")
print(f"P-value: {p_val}")

T-statistic: -4.511207179584193
P-value: 6.0323872941076986e-05


In [128]:
p_val < 0.05

True

In [129]:
np.mean(modelA_matches), np.mean(modelB_matches)

(24.5, 27.85)

Based on low p-value, we can reject the null hypothesis and accept the alternative hypothesis - model B (XGBoost) is better than model A (Neural Network).

Neither of the models achived the analytical success criteria of 80% match between real top tracks and the generated playlis.